In [ ]:
import pandas as pd

In [2]:
case_file = '/data100t1/home/egfrankel/anorexia/phecode_enrichment/AS_5_14_25_case_file_all_1yr_ehr.txt'
match_file = '/data100t1/home/egfrankel/anorexia/phecode_enrichment/AS_control_list_all.txt'
phecode_file = '/data100t1/home/egfrankel/anorexia/phecode_enrichment/AS_5_14_25_phecode_table_binary_all_1yr_ehr.txt.gz'

MGB_phecode = '/data100t1/home/egfrankel/anorexia/phecode_enrichment_mgb/all_output.frequencies_and_pvals_matched_forty_sig.txt'
Geisinger_phecode = '/data100t1/home/egfrankel/anorexia/Geisinger_Replication/all_output.frequencies_and_pvals_matched_forty_sig.txt'

In [14]:
case_control_df = pd.read_csv(match_file, sep='\t')
case_control_df.head()

,case,ctrl1,ctrl2,ctrl3,ctrl4,ctrl5,ctrl6
0,R200188025,R200021053,R200077671,R200126540,R200153504,R200153842,R200239055
1,R200209933,R200418141,R201836617,R202941179,R202977041,R203146499,R203200951
2,R200210788,R201758550,R202309143,R203197326,R203256698,R204464981,R205975267
3,R200270103,R200158590,R200217419,R200383764,R200503211,R200791195,R200827573
4,R200318300,R201998506,R202461963,R203351564,R203610850,R204328315,R204385014


In [15]:
# Rename columns ctrl1-6 to Control1-6
case_control_df = case_control_df.rename(
    columns={
        'ctrl1': 'Control1',
        'ctrl2': 'Control2',
        'ctrl3': 'Control3',
        'ctrl4': 'Control4',
        'ctrl5': 'Control5',
        'ctrl6': 'Control6'
    }
)

# Random split into 80% train, 20% test
case_control_df = case_control_df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle
split_idx = int(0.8 * len(case_control_df))
case_control_train = case_control_df.iloc[:split_idx].reset_index(drop=True)
case_control_test = case_control_df.iloc[split_idx:].reset_index(drop=True)

case_control_df.to_csv('../results/an_1203/case_control_pairs_an_1203.txt', sep='\t', index=False)
case_control_train.to_csv('../results/an_1203/case_control_pairs_train.txt', sep='\t', index=False)
case_control_test.to_csv('../results/an_1203/case_control_pairs_test.txt', sep='\t', index=False)

In [4]:
phecode_df = pd.read_csv(phecode_file, sep='\t')
phecode_df.head()

,grid,008,008.5,008.51,008.52,008.6,008.7,010,031,031.1,...,986,987,988,989,990,994,994.1,994.2,994.21,996
0,R200003427,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,R200003616,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,R200008325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,R200010803,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,R200014217,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
phecode_df.to_feather('../results/an_1203/binary_phecode.feather')

In [5]:
phecode_df.shape

(11574, 1878)

In [6]:
case_control_df.shape

(1676, 7)

In [16]:
enriched_phecode = pd.read_csv('../results/an_1203/an_an_1203_enriched_phecode.csv', sep='\t', dtype={'Phecode': str})
enriched_phecode.head()

,Phecode,Description,Count,p.value,p01,p05,p10,p50,p90,p95,p99,max,case_to_control_ratio
0,426.25,Other heart block,2,0.0,0,0,0,0,0,0,0,0,1000.0
1,985,Toxic effect of other metals,2,0.0,0,0,0,0,0,0,0,0,1000.0
2,117.2,Coccidioidomycosis,1,0.0,0,0,0,0,0,0,0,0,1000.0
3,452.8,Postphlebitic syndrome,1,0.0,0,0,0,0,0,0,0,0,1000.0
4,453,Chronic venous hypertension,1,0.0,0,0,0,0,0,0,0,0,1000.0


In [17]:
N_case = 1340

In [19]:
enriched_phecode['case_freq'] = enriched_phecode['Count'] / N_case
enriched_phecode_high_freq = enriched_phecode[enriched_phecode['case_freq'] > 0.0246]
enriched_phecode.shape, enriched_phecode_high_freq.shape

((706, 14), (349, 14))

In [25]:
enriched_phecode_high_freq = enriched_phecode_high_freq[enriched_phecode_high_freq['Phecode']!='305.21']
enriched_phecode_high_freq.shape

(348, 13)

In [ ]:
enriched_phecode_high_freq.drop(columns=['case_to_control_ratio'], inplace=True)
enriched_phecode_high_freq.to_csv('../results/an_1203/an_an_1203_enriched_phecode_high_freq.csv', sep='\t', index=False)

In [23]:
phecode_map = pd.read_csv('/data100t1/share/synthetic-deriv/phecodes/ICD_Phecode_mapping.csv', dtype=str)
phecode_map.head()

,ICD,Flag,ICDString,Phecode,PhecodeString,PhecodeCategory
0,001,9,Cholera,008,Intestinal infection,Intestinal infection
1,001.0,9,Cholera due to Vibrio cholerae,008,Intestinal infection,Intestinal infection
2,001.1,9,Cholera due to Vibrio cholerae el tor,008,Intestinal infection,Intestinal infection
3,001.9,9,Cholera NOS,008,Intestinal infection,Intestinal infection
4,002,9,Typhoid and paratyphoid fevers,008,Intestinal infection,Intestinal infection


In [24]:
an_icd_code = ['307.1', 'F50.00', 'F50.01', 'F50.02']
an_phecode_map = phecode_map[phecode_map['ICD'].isin(an_icd_code)]
an_phecode_map

,ICD,Flag,ICDString,Phecode,PhecodeString,PhecodeCategory
3694,307.1,9,Anorexia nervosa,305.21,Anorexia nervosa,psychological disorders
20605,F50.02,10,"Anorexia nervosa, binge eating/purging type",305.21,Anorexia nervosa,mental disorders
20606,F50.01,10,"Anorexia nervosa, restricting type",305.21,Anorexia nervosa,mental disorders
20608,F50.00,10,"Anorexia nervosa, unspecified",305.21,Anorexia nervosa,mental disorders


# Find overlapped enriched phecodes

In [3]:
mgd_phecode_df = pd.read_csv(MGB_phecode, sep='\t', dtype={'phecode': str})
mgd_phecode_df.head()

,phecode,pval,case_freq,phenotype,phecode_exclude_range,category_number,category
0,008,0,0.128099,Intestinal infection,001-009.99,1.0,infectious diseases
1,008.6,0,0.053719,Viral Enteritis,001-009.99,1.0,infectious diseases
2,038,0,0.099174,Septicemia,010-041.99,1.0,infectious diseases
3,038.3,0,0.078512,Bacteremia,010-041.99,1.0,infectious diseases
4,041,0,0.185950,Bacterial infection NOS,010-041.99,1.0,infectious diseases


In [4]:
gei_phecode_df = pd.read_csv(Geisinger_phecode, sep='\t', dtype={'phecode': str})
gei_phecode_df.head()

,phecode,pval,case_freq,phenotype,phecode_exclude_range,category_number,category
0,008,0,0.037175,Intestinal infection,001-009.99,1.0,infectious diseases
1,054,0,0.063197,Herpes simplex,050-079.99,1.0,infectious diseases
2,079,0,0.345725,Viral infection,050-079.99,1.0,infectious diseases
3,079.9,0,0.033457,"Viremia, NOS",050-079.99,1.0,infectious diseases
4,1002,0,0.171004,"Symptoms concerning nutrition, metabolism, and...",NaN,0.0,NaN


In [7]:
sd_phecode_df = pd.read_csv('/data100t1/home/biand/Projects/PheWES/results/an_1203/an_an_1203_enriched_phecode_high_freq.csv', sep='\t', dtype={'Phecode': str})
sd_phecode_df.head()

,Phecode,Description,Count,p.value,p01,p05,p10,p50,p90,p95,p99,max,case_freq
0,297,Suicidal ideation or attempt,330,0.0,0,0,0,0,0,0,0,0,0.246269
1,297.1,Suicidal ideation,294,0.0,0,0,0,0,0,0,0,0,0.219403
2,295,Schizophrenia and other psychotic disorders,59,0.0,0,0,0,0,0,0,0,0,0.044030
3,306.9,Tension headache,37,0.0,0,0,0,0,0,0,0,0,0.027612
4,260,Protein-calorie malnutrition,761,0.0,0,0,0,0,0,0,0,0,0.567910


In [8]:
mgd_phecode_df.shape, gei_phecode_df.shape, sd_phecode_df.shape

((685, 7), (369, 7), (348, 13))

In [9]:
overlapped_phecode = set(mgd_phecode_df.phecode.to_list()) & set(gei_phecode_df.phecode.to_list()) & set(sd_phecode_df.Phecode.to_list())

In [10]:
len(overlapped_phecode)

209

In [11]:
overlapped_phecode

{'008',
 '079',
 '079.9',
 '1002',
 '1005',
 '1009',
 '1010',
 '1011',
 '1015',
 '110.1',
 '1100',
 '112',
 '216',
 '244.4',
 '246',
 '246.7',
 '250.2',
 '250.42',
 '251.1',
 '260',
 '260.1',
 '260.2',
 '260.6',
 '260.7',
 '261.2',
 '261.4',
 '263',
 '272.1',
 '275.53',
 '276.1',
 '276.12',
 '276.14',
 '276.5',
 '277',
 '278',
 '278.1',
 '278.4',
 '280.1',
 '285',
 '288',
 '288.1',
 '288.2',
 '289',
 '291.8',
 '292',
 '296',
 '296.1',
 '296.2',
 '296.22',
 '297',
 '297.1',
 '297.2',
 '300',
 '300.1',
 '300.11',
 '300.12',
 '300.3',
 '300.4',
 '300.8',
 '300.9',
 '301',
 '301.2',
 '303.4',
 '304',
 '305.2',
 '306',
 '312',
 '313',
 '313.1',
 '315',
 '316',
 '317',
 '317.1',
 '318',
 '327',
 '327.3',
 '327.32',
 '327.4',
 '333',
 '338.2',
 '339',
 '340',
 '340.1',
 '345',
 '345.3',
 '350.1',
 '350.2',
 '351',
 '355.1',
 '367',
 '368',
 '369.5',
 '380.1',
 '380.4',
 '381.1',
 '381.11',
 '382',
 '386.9',
 '389',
 '401.1',
 '418',
 '426.7',
 '427.3',
 '427.7',
 '427.9',
 '429.3',
 '455',
 '

In [12]:
sd_phecode_filtered_df = sd_phecode_df[sd_phecode_df.Phecode.isin(overlapped_phecode)]

In [13]:
sd_phecode_filtered_df.to_csv('../results/an_1203/an_an_1203_enriched_phecode.csv', sep='\t', index=False)